In [1]:
# yale face

In [2]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.decomposition import KernelPCA, PCA, NMF
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    pairwise_distances,
    mean_squared_error,
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
yale_faces = pd.DataFrame()

for file in os.listdir("yalefacespng"):
    filename = file.split(".")[0]
    yale_faces = pd.concat(
        [
            yale_faces,
            pd.DataFrame(
                np.array(Image.open(f"yalefacespng\\{file}")).flatten(),
                columns=[f"{filename}"],
            ),
        ],
        axis=1,
    )
yale_faces = yale_faces.T.reset_index()
yale_faces["target"] = yale_faces["index"].map(lambda filename: int(filename[1:3]))
yale_faces["expression"] = yale_faces["index"].map(lambda filename: filename[3:])
yale_faces = yale_faces.drop(columns=["index"])
yale_faces.head(12)

image_data = np.array(yale_faces.drop(columns=["target", "expression"], axis=1))
image_labels = np.array(yale_faces["target"])

display(image_data)
display(image_labels)

array([[130, 130, 130, ...,  68,  68,  68],
       [130, 130, 130, ...,  68,  68,  68],
       [130, 130, 130, ...,  68,  68,  68],
       ...,
       [108, 116, 117, ...,  68,  68,  68],
       [130, 130, 130, ...,  68,  68,  68],
       [130, 130, 130, ...,  68,  68,  68]],
      shape=(90, 77760), dtype=uint8)

array([ 1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  9,  9,  9,
        9,  9,  9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12,
       12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15,
       15, 15, 15, 15, 15])

In [ ]:
# Question 1
# norm split
yale_faces_norm = yale_faces[yale_faces["expression"] == "norm"].drop(
    columns=["expression"], axis=1
)

# other split
yale_faces_other = yale_faces[yale_faces["expression"] != "norm"].drop(
    columns=["expression"], axis=1
)

In [ ]:
# Question 1
pred_norm = []

for subject in range(yale_faces_other.shape[0]):
    target = yale_faces_other.iloc[subject].target
    sub_img = yale_faces_other.iloc[subject].drop("target")

    min_dist = np.inf
    best_norm_sub = None

    for norm_subject in range(yale_faces_norm.shape[0]):
        norm = yale_faces_norm.iloc[norm_subject].target
        norm_sub_img = yale_faces_norm.iloc[norm_subject].drop("target")

        dist = np.linalg.norm(sub_img - norm_sub_img)
        best_norm_sub = norm if dist < min_dist else best_norm_sub
        min_dist = dist if dist < min_dist else min_dist

    print([subject, target, best_norm_sub])
    pred_norm.append([subject, target, best_norm_sub])

In [ ]:
# Question 1

pred_norm_df = pd.DataFrame(pred_norm, columns=["subject", "target", "predicted"])
pred_norm_df["correct"] = pred_norm_df["target"] == pred_norm_df["predicted"]
(
    pred_norm_df["correct"].sum() / pred_norm_df["correct"].count(),
    pred_norm_df["correct"].sum(),
    pred_norm_df["correct"].count(),
)

In [6]:
# Question 2 param concat

image_data_df = pd.DataFrame(image_data)
image_labels_df = pd.DataFrame(image_labels, columns=["target"])

images_df = pd.concat([image_data_df, image_labels_df], axis=1)

In [8]:
# Question 2

subject_reps_2 = pd.DataFrame()
for subject_number in images_df["target"].unique():
    subject_images = (
        images_df[images_df["target"] == subject_number]
        .drop(columns=["target"], axis=1)
        .T
    )
    pca = PCA(n_components=1)
    subject_images_lr = pd.DataFrame(
        pca.inverse_transform(pca.fit_transform(subject_images))
    ).T
    subject_rep = pd.DataFrame(subject_images_lr.mean(axis=0)).T
    subject_rep["target"] = subject_number
    subject_reps_2 = pd.concat([subject_reps_2, subject_rep])

subject_reps_2

,0,1,2,3,4,5,6,7,8,9,...,77751,77752,77753,77754,77755,77756,77757,77758,77759,target
0,131.641142,131.641142,131.641142,131.641142,131.641142,131.641142,131.641142,131.641142,131.641142,131.466928,...,70.228480,70.228480,70.228480,70.228480,70.228480,70.228480,70.228480,70.228480,70.228480,1
0,115.835448,117.214872,126.021796,129.274017,127.560549,126.870837,129.804399,130.319397,129.789015,128.939973,...,68.600029,68.600029,68.600029,68.600029,68.600029,68.600029,68.600029,68.600029,68.600029,2
0,129.007422,129.007422,129.007422,129.007422,129.007422,129.007422,129.007422,129.007422,129.007422,129.007422,...,67.349366,67.349366,67.349366,67.349366,67.349366,67.349366,67.349366,67.349366,67.349366,3
0,129.572470,129.572470,129.572470,129.572470,129.572470,129.572470,129.572470,129.572470,129.572470,129.572470,...,67.874770,67.874770,67.874770,67.874770,67.874770,67.874770,67.874770,67.874770,67.874770,4
0,121.450771,128.832354,129.518794,129.518794,129.518794,129.518794,128.316743,128.489854,129.518794,129.518794,...,67.720157,67.720157,67.720157,67.720157,67.720157,67.720157,67.720157,67.720157,67.720157,5
0,116.310394,106.250676,102.474215,114.873291,121.340252,124.049743,130.127582,130.846134,130.846134,130.846134,...,77.029497,77.029497,77.029497,77.029497,77.029497,77.029497,77.029497,77.029497,77.029497,6
0,128.915873,128.915873,128.915873,128.915873,128.915873,128.915873,128.915873,128.915873,128.915873,128.915873,...,67.264735,67.264735,67.264735,67.264735,67.264735,67.264735,67.264735,67.264735,67.264735,7
0,128.495047,128.495047,128.495047,128.495047,128.495047,128.495047,128.495047,128.495047,128.495047,128.495047,...,67.215581,67.215581,67.215581,67.215581,67.215581,67.215581,67.215581,67.215581,67.215581,8
0,129.571828,129.571828,129.571828,129.571828,129.571828,129.571828,129.571828,129.571828,129.571828,129.571828,...,68.648746,68.648746,68.648746,68.648746,68.648746,68.648746,68.648746,68.648746,68.648746,9
0,129.418145,129.418145,129.418145,129.418145,129.418145,129.418145,129.418145,129.418145,129.418145,129.418145,...,67.984397,67.984397,67.984397,67.984397,67.984397,67.984397,67.984397,67.984397,67.984397,10


In [9]:
# Question 2
pred_rep_2 = []

for subject in range(images_df.shape[0]):
    target = images_df.iloc[subject].target
    sub_img = images_df.iloc[subject].drop("target")

    min_dist = np.inf
    best_norm_sub = None

    for norm_subject in range(subject_reps_2.shape[0]):
        norm = subject_reps_2.iloc[norm_subject].target
        norm_sub_img = subject_reps_2.iloc[norm_subject].drop("target")

        dist = np.linalg.norm(sub_img - norm_sub_img)
        best_norm_sub = norm if dist < min_dist else best_norm_sub
        min_dist = dist if dist < min_dist else min_dist

    print([subject, target, best_norm_sub])
    pred_rep_2.append([subject, target, best_norm_sub])

[0, np.int64(1), np.float64(1.0)]
[1, np.int64(1), np.float64(1.0)]
[2, np.int64(1), np.float64(1.0)]
[3, np.int64(1), np.float64(12.0)]
[4, np.int64(1), np.float64(1.0)]
[5, np.int64(1), np.float64(1.0)]
[6, np.int64(2), np.float64(2.0)]
[7, np.int64(2), np.float64(2.0)]
[8, np.int64(2), np.float64(2.0)]
[9, np.int64(2), np.float64(2.0)]
[10, np.int64(2), np.float64(2.0)]
[11, np.int64(2), np.float64(2.0)]
[12, np.int64(3), np.float64(3.0)]
[13, np.int64(3), np.float64(3.0)]
[14, np.int64(3), np.float64(3.0)]
[15, np.int64(3), np.float64(12.0)]
[16, np.int64(3), np.float64(3.0)]
[17, np.int64(3), np.float64(3.0)]
[18, np.int64(4), np.float64(4.0)]
[19, np.int64(4), np.float64(4.0)]
[20, np.int64(4), np.float64(4.0)]
[21, np.int64(4), np.float64(4.0)]
[22, np.int64(4), np.float64(4.0)]
[23, np.int64(4), np.float64(4.0)]
[24, np.int64(5), np.float64(5.0)]
[25, np.int64(5), np.float64(5.0)]
[26, np.int64(5), np.float64(5.0)]
[27, np.int64(5), np.float64(12.0)]
[28, np.int64(5), np.float6

In [10]:
# Question 2
pred_rep_2_df = pd.DataFrame(pred_rep_2, columns=["subject", "target", "predicted"])
pred_rep_2_df["correct"] = pred_rep_2_df["target"] == pred_rep_2_df["predicted"]
(
    pred_rep_2_df["correct"].sum() / pred_rep_2_df["correct"].count(),
    pred_rep_2_df["correct"].sum(),
    pred_rep_2_df["correct"].count(),
)

(np.float64(0.8555555555555555), np.int64(77), np.int64(90))

In [11]:
# Code submission
def predict_class(image_number, image_data, image_labels):

    image_data_df = pd.DataFrame(image_data)
    image_labels_df = pd.DataFrame(image_labels, columns=["target"])

    images_df = pd.concat([image_data_df, image_labels_df], axis=1)

    subject_reps_2 = pd.DataFrame()
    for subject_number in images_df["target"].unique():
        subject_images = (
            images_df[images_df["target"] == subject_number]
            .drop(columns=["target"], axis=1)
            .T
        )
        pca = PCA(n_components=1)
        subject_images_lr = pd.DataFrame(
            pca.inverse_transform(pca.fit_transform(subject_images))
        ).T
        subject_rep = pd.DataFrame(subject_images_lr.mean(axis=0)).T
        subject_rep["target"] = subject_number
        subject_reps_2 = pd.concat([subject_reps_2, subject_rep])

    target = images_df.iloc[image_number - 1].target
    sub_img = images_df.iloc[image_number - 1].drop("target")

    min_dist = np.inf
    best_norm_sub = None

    for norm_subject in range(subject_reps_2.shape[0]):
        norm = subject_reps_2.iloc[norm_subject].target
        norm_sub_img = subject_reps_2.iloc[norm_subject].drop("target")

        dist = np.linalg.norm(sub_img - norm_sub_img)
        best_norm_sub = norm if dist < min_dist else best_norm_sub
        min_dist = dist if dist < min_dist else min_dist
    return best_norm_sub

In [ ]:
predict_class(64, image_data, image_labels)

np.float64(11.0)

In [ ]:
# Question 3
subject_nmf_reps = pd.DataFrame()
for subject_no in yale_faces["target"].unique():
    subject_images = yale_faces[yale_faces["target"] == subject_no].drop(
        columns=["expression", "target"], axis=1
    )

    nmf_model = NMF(n_components=1, random_state=42, init='random')
    W = nmf_model.fit_transform(subject_images)  # Relationship of images to latent features
    subject_images_lr = nmf_model.inverse_transform(W)
    subject_rep = pd.DataFrame(subject_images_lr.mean(axis=0)).T
    subject_rep["target"] = subject_no
    subject_nmf_reps = pd.concat([subject_nmf_reps, subject_rep])

subject_nmf_reps

In [ ]:
# Question 3
pred_nmf = []

for subject in range(yale_faces.shape[0]):
    target = yale_faces.iloc[subject].target
    sub_img = yale_faces.iloc[subject].drop("target").drop('expression')

    min_dist = np.inf
    best_nmf_sub = None

    for nmf_subject in range(subject_nmf_reps.shape[0]):
        nmf_target = subject_nmf_reps.iloc[nmf_subject].target
        nmf_sub_img = subject_nmf_reps.iloc[nmf_subject].drop("target")

        dist = np.linalg.norm(sub_img - nmf_sub_img)
        best_nmf_sub = nmf_target if dist < min_dist else best_nmf_sub
        min_dist = dist if dist < min_dist else min_dist

    print([subject, target, best_nmf_sub])
    pred_nmf.append([subject, target, best_nmf_sub])

In [ ]:
# Question 3
pred_nmf_df = pd.DataFrame(pred_nmf, columns=["subject", "target", "predicted"])
pred_nmf_df["correct"] = pred_nmf_df["target"] == pred_nmf_df["predicted"]
(
    pred_nmf_df["correct"].sum() / pred_nmf_df["correct"].count(),
    pred_nmf_df["correct"].sum(),
    pred_nmf_df["correct"].count(),
)